In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/GOVHACK

/content/drive/MyDrive/GOVHACK


In [3]:
ls

 codes/
'Combined datasets used for the poewer BI Dashboard'/
'DATA SET FOR Delivering the 20-Minute Neighbourhood Plan'/
'DATA SET FOR Enabling Better Community Housing and Infrastructure Planning'/
'Gov hack 25.docx'
 no_rows_found.png
'what to do.docx'


In [4]:
!pip -q install folium pandas


In [19]:
from pathlib import Path
import re
import pandas as pd
import folium
from folium import Map
from folium.features import DivIcon
import json
from branca.element import Template, MacroElement

In [20]:
# ---- INPUT / OUTPUT ----
CSV_PATH = Path("DATA SET FOR Enabling Better Community Housing and Infrastructure Planning/Vic Government Data - Building Permits/20251496-Rawdata-July-20251_with_coords.csv")
OUT_HTML = Path("DATA SET FOR Enabling Better Community Housing and Infrastructure Planning/Vic Government Data - Building Permits/building_permits_map.html")  # saved map

# ---- COLUMN NAMES IN YOUR CSV ----
COL_STREET   = "Site_street_name"
COL_SUBURB   = "site_town_suburb__c"
COL_POSTCODE = "site_postcode__c"
COL_USE      = "BASIS_Building_Use"
COL_LON      = "coordinates x"   # x = lon
COL_LAT      = "coordinates y"   # y = lat

# Fixed palette (canonical 7)
USE_PALETTE = {
    "Commercial":            "#1f77b4",
    "Domestic":              "#ff7f0e",
    "Hospital/Healthcare":   "#2ca02c",
    "Industrial":            "#d62728",
    "Public Buildings":      "#9467bd",
    "Residential":           "#8c564b",
    "Retail":                "#17becf",
}

In [ ]:
class BuildingPermitsMap:
    def __init__(self, csv_path: Path):
        self.csv_path = Path(csv_path)

    # map only to the 7 canonical labels; return None if no match (row will be dropped)
    @staticmethod
    def _canon_use(v: str):
        t = re.sub(r"\s+", " ", str(v or "")).strip().lower()
        if t.startswith("comm"):        return "Commercial"
        if t.startswith("domestic"):    return "Domestic"
        if "hospital" in t or "health" in t: return "Hospital/Healthcare"
        if t.startswith("indus"):       return "Industrial"
        if t.startswith("public"):      return "Public Buildings"
        if t.startswith("resid"):       return "Residential"
        if t.startswith("retail"):      return "Retail"
        return None  # <- drop anything else

    @staticmethod
    def _popup(row) -> str:
        def g(col):
            return "" if col not in row or pd.isna(row[col]) else str(row[col])
        return (
            f"<b>BASIS_Building_Use:</b> {g(COL_USE)}"
            f"<br><b>Street:</b> {g(COL_STREET)}"
            f"<br><b>Suburb:</b> {g(COL_SUBURB)}"
            f"<br><b>Postcode:</b> {g(COL_POSTCODE)}"
            f"<br><b>Lon/Lat:</b> {g(COL_LON)}, {g(COL_LAT)}"
        )

    def build(self) -> Map:
        df = pd.read_csv(self.csv_path, sep=None, engine="python")
        df.columns = [c.strip() for c in df.columns]
        df[COL_LON] = pd.to_numeric(df[COL_LON], errors="coerce")
        df[COL_LAT] = pd.to_numeric(df[COL_LAT], errors="coerce")
        df = df.dropna(subset=[COL_LON, COL_LAT]).copy()

        # canonicalize to 7 types and drop everything else
        df[COL_USE] = df[COL_USE].apply(self._canon_use)
        df = df[df[COL_USE].notna()].copy()

        # keep only colors for present categories
        present = sorted(df[COL_USE].unique().tolist())
        color_map = {k: USE_PALETTE[k] for k in present}

        center = [df[COL_LAT].mean(), df[COL_LON].mean()]
        m = folium.Map(location=center, zoom_start=9, tiles="CartoDB positron")

        # one FeatureGroup per type (togglable)
        groups = {use: folium.FeatureGroup(name=use, show=True) for use in present}
        for _, r in df.iterrows():
            use = r[COL_USE]
            grp = groups[use]
            color = color_map[use]
            folium.CircleMarker(
                location=(r[COL_LAT], r[COL_LON]),
                radius=5, weight=1,
                color=color, fill=True, fill_color=color, fill_opacity=0.9,
                popup=folium.Popup(self._popup(r), max_width=360),
            ).add_to(grp)

        for grp in groups.values():
            grp.add_to(m)

        # Layer toggle (top-right)
        folium.LayerControl(position="topright", collapsed=False).add_to(m)

        # Inject a legend INSIDE the LayerControl, placed UNDER the checkbox list
        colors_json = json.dumps(color_map)
        js = """
        <script>
        (function() {
          const COLORS = __COLORS__;
          function paint() {
            const labels = document.querySelectorAll('.leaflet-control-layers-overlays label');
            labels.forEach(l => {
              const name = l.textContent.trim();
              const color = COLORS[name];
              if (!color) return;
              if (l.querySelector('.legend-chip')) return;
              const chip = document.createElement('span');
              chip.className = 'legend-chip';
              chip.style.cssText =
                'display:inline-block;width:12px;height:12px;margin:0 6px;'
                + 'border:1px solid #555;background:'+color+';';
              const cb = l.querySelector('input[type="checkbox"]');
              if (cb) cb.after(chip); else l.prepend(chip);
            });
          }
          setTimeout(paint, 300);
          const ctl = document.querySelector('.leaflet-control-layers') || document.body;
          new MutationObserver(paint).observe(ctl, {childList:true, subtree:true});
        })();
        </script>
        """.replace("__COLORS__", colors_json)

        _swatches = MacroElement()
        _swatches._template = Template(js)
        m.get_root().add_child(_swatches)

        return m

    def save(self, m: Map, out_html: Path) -> Path:
        out_html = Path(out_html)
        m.save(str(out_html))
        print(f"Saved map → {out_html}")
        return out_html

# -------- run --------
builder = BuildingPermitsMap(CSV_PATH)
m = builder.build()
m  # display in Colab

# # also save to file beside the notebook
# builder.save(m, OUT_HTML)
